In [ ]:
import pandas as pd
import requests
import json
import numpy as np

In [ ]:
def getAllStreamStats(i):
    ## Define gage location
#     y_loc = dat["dec_lat_va"][i]
#     x_loc = dat["dec_long_va"][i]
    
    y_loc = np.round(dat["dec_lat_va"][i],2)
    x_loc = np.round(dat["dec_long_va"][i],2)
    state = 'OR'
    gage = dat["gage"][i]
    
    
    ## Define Streamstats API URLS
    # Service URLS
    StreamStatsServiceURLS = {
        'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
        'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
        'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
        }
    NSSServiceURlS = {
        'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
        'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
        'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
    }

    # Declare cookies for use of consistent StreamStats servers
    global cookies 
    
    # Step 1 - Get the watershed
    print('Step 1 - Get the watershed')
    watershedURLParams = {
        'rcode': state, 
        'xlocation': x_loc, 
        'ylocation': y_loc,
        'crs': 4326,
    #     'crs': 4269,
    }
    delineatedBasin = None
    while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
        print('Delineating basin...')
        basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
        if basinDelineationResponse.status_code == 200:
            cookies = basinDelineationResponse.cookies
            try:
                delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
                workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
            except:
                print("No geometry returned. Retrying...")
                pass
        else:
            print("Error. Retrying...")
            pass

    # Step 2 - Get the Regression Regions associated with the watershed
    print('Step 2 - Get the Regression Regions associated with the watershed')
    regressionRegionPOSTBody = delineatedBasin # from step 1

    regressionRegions = None
    while regressionRegions == None:
        regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
        if regressionRegionsResponse.status_code == 200:
            regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
            regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
        else:
            print("Error.")
            pass

    # Step 3 - Get the Scenarios associated with the Regression Regions
    print('Step 3 - Get the Scenarios associated with the Regression Regions')

    scenarioURLParams = {
        'regions': state, 
        'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
        'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
    }
    scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

    parameters = None
    while parameters == None:
        if scenarioResponse.status_code == 200:
            scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
            parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
            parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
        else:
            print("Error.")
            pass

    # Step 4 - Compute the basin characteristics
    print('Step 4 - Compute the basin characteristics')
    basinCharacteristicsURLParams = {
        'rcode': state, 
        'workspaceID': workspaceID, # from step 1
        'includeparameters': (','.join(parameterCodes)) # from step 3
    }

    basinCharacteristics = None
    while basinCharacteristics is None:
        basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
        if basinCharacteristicsResponse.status_code == 200:
            basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
        else:
            print("Error.")
            pass

    # Step 5 - Compute Flow Statistics
    print('Step 5 - Compute Flow Statistics')
    flowStatsURLParams = {
        'regions': state
    }
    flowStatsPOSTBody = [] 
    for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
        for p in basinCharacteristics['parameters']:
            if x['code'].lower() == p['code'].lower():
                scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
    flowStatsPOSTBody.append(scenarios)

    flowStats = None
    while flowStats == None:
        flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
        if flowStatsResponse.status_code == 200:
            flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
        else:
            print("Error.")
            pass

    out = pd.json_normalize(flowStats,
                             record_path=['regressionRegions',['results']],
                            )[["code","value"]]
    out["gage"] = gage
    return out

In [327]:
pnwNPall = pd.read_csv("../data/pnwNPall_InfowStats.csv")
siteID = list(pnwNPall['gage'].astype(str))
dat = nwis.get_info(sites = siteID)[0]


dat["state"] = "none"
dat["state"][dat["state_cd"]==16] = "ID"
dat["state"][dat["state_cd"]==41] = "OR"
dat["state"][dat["state_cd"]==53] = "WA"

st = ["ID",'WA','OR']
dat = dat[dat["state"].isin(st)].reset_index(drop=True)
dat = dat[["site_no","dec_lat_va","dec_long_va",'state','station_nm']]
dat.columns = ["gage","lat","long",'state','name']
dat['gage']  =dat['gage'].astype(int)
dat["lat"] = round(dat["lat"],3)
dat["long"] = round(dat["long"],3)

<ipython-input-327-2de0735adb11>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["state"][dat["state_cd"]==16] = "ID"
<ipython-input-327-2de0735adb11>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["state"][dat["state_cd"]==41] = "OR"
<ipython-input-327-2de0735adb11>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat["state"][dat["state_cd"]==53] = "WA"


In [328]:
dat[dat["gage"]==12115700]

,gage,lat,long,state,name
61,12115700,47.366,-121.693,WA,"BOULDER CREEK NEAR CEDAR FALLS, WA"


In [366]:
i = 4


## Define gage location
y_loc = dat["lat"][i]
x_loc = dat["long"][i]
state = dat["state"][i]
gage = dat["gage"][i]


## Define Streamstats API URLS
# Service URLS
StreamStatsServiceURLS = {
    'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
    'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
    }
NSSServiceURlS = {
    'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
    'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
    'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
}

# Declare cookies for use of consistent StreamStats servers
global cookies 

# Step 1 - Get the watershed
print('Step 1 - Get the watershed')
watershedURLParams = {
    'rcode': state, 
    'xlocation': x_loc, 
    'ylocation': y_loc,
    'crs': 4326,
#     'crs': 4269,
}
delineatedBasin = None
while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
    print('Delineating basin...')
    basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
    if basinDelineationResponse.status_code == 200:
        cookies = basinDelineationResponse.cookies
        try:
            delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
            workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
        except:
            print("No geometry returned. Retrying...")
    else:
        print("Error. Retrying...")

# Step 2 - Get the Regression Regions associated with the watershed
print('Step 2 - Get the Regression Regions associated with the watershed')
regressionRegionPOSTBody = delineatedBasin # from step 1

regressionRegions = None
while regressionRegions == None:
    regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
    if regressionRegionsResponse.status_code == 200:
        regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
        regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
    else:
        print("Error.")
        pass

# Step 3 - Get the Scenarios associated with the Regression Regions
print('Step 3 - Get the Scenarios associated with the Regression Regions')

scenarioURLParams = {
    'regions': state, 
    'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
    'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
}
scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

parameters = None
while parameters == None:
    if scenarioResponse.status_code == 200:
        scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
        parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
        parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
    else:
        print("Error.")
        pass

# Step 4 - Compute the basin characteristics
print('Step 4 - Compute the basin characteristics')
basinCharacteristicsURLParams = {
    'rcode': state, 
    'workspaceID': workspaceID, # from step 1
    'includeparameters': (','.join(parameterCodes)) # from step 3
}

basinCharacteristics = None
while basinCharacteristics is None:
    basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
    if basinCharacteristicsResponse.status_code == 200:
        basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
    else:
        print("Error.")
        pass

# Step 5 - Compute Flow Statistics
print('Step 5 - Compute Flow Statistics')
flowStatsURLParams = {
    'regions': state
}
flowStatsPOSTBody = [] 
for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
    for p in basinCharacteristics['parameters']:
        if x['code'].lower() == p['code'].lower():
            scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
flowStatsPOSTBody.append(scenarios)

flowStats = None
while flowStats == None:
    flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
    if flowStatsResponse.status_code == 200:
        flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
    else:
        print("Error.")
        pass

out = pd.json_normalize(flowStats,
                         record_path=['regressionRegions',['results']],
                        )[["code","value","equation"]]
out["gage"] = gage

Step 1 - Get the watershed
Delineating basin...
Step 2 - Get the Regression Regions associated with the watershed
Step 3 - Get the Scenarios associated with the Regression Regions
Step 4 - Compute the basin characteristics
Step 5 - Compute Flow Statistics


In [386]:
METRIC = "M7D10Y"
eq = out['equation'][out["code"]==METRIC].values[0].replace("^", "**")
TT = eval(eq)

In [256]:
StreamStatsServiceURLS = {
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/characteristics'
    }


In [399]:
i=61

gageURLParams = {
    "stationIDOrCode": dat["gage"][i]
}


gageResponse = requests.get(url = StreamStatsServiceURLS['gage'], params = gageURLParams)

if gageResponse.status_code == 200:
    cookies = gageResponse.cookies
    try:
        gageStats = pd.DataFrame(json.loads(gageResponse.content.decode('utf-8')))


        varCode = pd.DataFrame(list(gageStats["variableType"]))["code"]
        units = pd.DataFrame(list(gageStats["unitType"]))["name"]

        gS = pd.DataFrame(gageStats[["variableTypeID","value"]]).index_reset(drop=True)
        gS["code"] = varCode
        gS["unit"] = units
#         gS =gS.set_index('code')
    except:
        pass
    
else:
    pass

In [316]:
precipCHECK = basinCharacteristics["parameters"][1]["code"]
PRECIP = basinCharacteristics["parameters"][1]["value"]
DRNAREA = gS["value"][gS['code'] == "DRNAREA"].values

In [323]:
gS['code'].values

array(['I24H2Y', 'I24H50Y', 'I24H100Y', 'DRNAREA', 'LAT_CENT',
       'LONG_CENT', 'ELEVMAX', 'ELEV', 'BSHAPE', 'JANMINTMP', 'MINBELEV',
       'RELIEF', 'STRMTOT', 'NFSL30_30M', 'SLOP30_30M', 'BSLDEM30M',
       'IMPNLCD01', 'BASLENAH', 'DRNDENSITY', 'PRECPRIS10', 'LC01CANOPY',
       'SSURGINDEX'], dtype=object)

In [388]:
# out = pd.json_normalize(flowStats,
#                          record_path=['regressionRegions',['results']],
#                         )[["code","value","equation"]]



In [401]:
"]

KeyError: 0